In [3]:
import pandas as pd
import numpy as np

from random import randint
from numpy import array_equal
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import RepeatVector




In [4]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

train = pd.read_csv('train.csv')



In [5]:
import pandas as pd
import numpy as np
import re 
from keras.layers import Input, Embedding, LSTM, Dense, RepeatVector, Bidirectional, Dropout
#from keras.optimizers import Adam, SGD
from keras.models import Model
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
#from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint
from collections import Counter
import nltk

In [6]:
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed
[nltk_data]     (_ssl.c:777)>


False

In [7]:
from nltk.corpus import stopwords


In [8]:
from nltk.stem.porter import PorterStemmer


In [9]:
ps=PorterStemmer()

In [10]:
train.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [11]:
train.shape

(159571, 8)

In [12]:
X = train['comment_text']
X[1]

"D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)"

In [13]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'(\w)\1{2,}', r'\1\1', text)
    text = re.sub(r'(\W)\1+', r'\1', text)
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    text=re.sub('[^a-zA-Z]',' ',text)
    text=text.lower()
    text=text.split()
    text=[ps.stem(word) for word in text if not word in set(stopwords.words('english'))]
    text=' '.join(text)
    
    
    
    return text

In [ ]:
clean_questions = []
for question in X:
    
    clean_questions.append(clean_text(question))
    

In [13]:
word2em = dict()
f = open('glove.6B.50d.txt',encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word2em[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(word2em))


Loaded 400000 word vectors.


In [14]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [15]:

t = Tokenizer()
t.fit_on_texts(clean_questions)
vocab_size = len(t.word_index) + 1


In [16]:
vocab_size

181609

In [17]:
encoded_docs = t.texts_to_sequences(clean_questions)
print(encoded_docs[1])

[5952, 880, 1159, 2051, 3325, 2094, 12, 7, 817, 117]


In [18]:
max_length = 50
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs[10])

[ 423   46  143   77    6  280  371   96  153  697   13   77   63   46  371
  280  138   30  992 3256   47   26   11    9  366   77  351  289  182   11
   47  143 1145  579  401 2695    6   47   11  476   54   10   70  281  143
   54    2   12    7  397]


In [19]:
embedding_matrix = np.zeros((vocab_size, 50))
for word, i in t.word_index.items():
    embedding_vector = word2em.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [20]:
embedding_matrix.shape

(181609, 50)

In [29]:
from keras.layers import Bidirectional, GlobalMaxPool1D
inp = Input(shape=(50,))
x = Embedding(vocab_size, 50, weights=[embedding_matrix],trainable=False,mask_zero=False)(inp)
x = Bidirectional(LSTM(128, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = Bidirectional(LSTM(256, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)


x = GlobalMaxPool1D()(x)
x = Dense(100, activation="sigmoid")(x)
x = Dropout(0.2)(x)

x = Dense(6, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [2]:
from keras.layers import Dense, Input, LSTM, GRU, Embedding, Dropout, Activation, BatchNormalization
from keras.layers import Bidirectional, GlobalMaxPool1D
inp = Input(shape=(50,))
x = Embedding(vocab_size, 50, weights=[embedding_matrix],trainable=True,mask_zero=False)(inp)
x = Dropout(0.2)(x)
x = Bidirectional(LSTM(140, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)


x = GlobalMaxPool1D()(x)
x = BatchNormalization()(x)

x = Dense(50, activation="sigmoid")(x)
x = Dropout(0.17)(x)

x = Dense(6, activation="sigmoid")(x)
model4 = Model(inputs=inp, outputs=x)
model4.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Using TensorFlow backend.


NameError: name 'vocab_size' is not defined

In [23]:
y = train.iloc[:,2:8].values

In [24]:
y.shape

(159571, 6)

In [43]:
model.fit(padded_docs, y, batch_size=32, epochs=2)

Epoch 1/2
159571/159571 [==============================] - 2234s 14ms/step - loss: 0.0518 - acc: 0.9817
Epoch 2/2
159571/159571 [==============================] - 2115s 13ms/step - loss: 0.0495 - acc: 0.9823


In [55]:
model2.fit(padded_docs, y, batch_size=64, epochs=2)

Epoch 1/2
159571/159571 [==============================] - 608s 4ms/step - loss: 0.0241 - acc: 0.9904
Epoch 2/2
159571/159571 [==============================] - 627s 4ms/step - loss: 0.0220 - acc: 0.9915


In [1]:
model4.fit(padded_docs, y, batch_size=64, epochs=7)

NameError: name 'model4' is not defined

In [42]:

model.save('model_toxic_final_2.h5')


In [65]:
model3.save('model_toxic_final_bestm3.h5')

In [28]:
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

In [29]:
test = pd.read_csv('test.csv')

In [30]:
Y = test.iloc[:,1]


In [31]:
clean_questions1 = []
for question in Y:
    clean_questions1.append(clean_text(question))

In [32]:
vocab_size

181609

In [33]:
encoded_docs1 = t.texts_to_sequences(clean_questions1)
print(encoded_docs1[1])

[1044, 220, 542, 2140]


In [34]:
padded_docs1 = pad_sequences(encoded_docs1, maxlen=max_length, padding='post')
print(padded_docs1[10])

[  15 1169   30    1 1014   18   55   44    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0]


In [35]:
embedding_matrix1 = np.zeros((vocab_size, 50))
for word, i in t.word_index.items():
    embedding_vector = word2em.get(word)
    if embedding_vector is not None:
        embedding_matrix1[i] = embedding_vector

In [72]:
y_pred = model4.predict([padded_docs1], batch_size=1000, verbose=1)

153164/153164 [==============================] - 79s 516us/step


In [73]:
#6 was best
sample_submission = pd.read_csv('sample_submission.csv')
sample_submission[list_classes] = y_pred
sample_submission.to_csv('submission_final_today6.csv', index=False)

In [49]:
check.shape

(153164, 7)

In [21]:
from keras.models import load_model

In [23]:
model=load_model('model_toxic_final.h5')